# Step 02: Group Rollup (Groups of Groups)

This notebook submits rollup grouping jobs to Moody's Risk Modeler for **groups that contain other groups**.

Rollup groups contain references to OTHER groups (not just analyses). These can only be created AFTER the child groups from Step_01 have been created.

**Tasks:**
- Verify prerequisite: Grouping batch (Step_01) must be COMPLETED
- Retrieve Grouping Rollup batch from Stage_01/Step_03
- Review rollup job configurations
- Submit rollup jobs to Moody's API
- Track job completion status

**IMPORTANT:** This notebook will fail if the Grouping batch from Step_01 is not complete.

## 1) Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.batch import submit_batch, get_batch_jobs, read_batch
from helpers.database import execute_query
from helpers.step import get_last_step_run
from helpers.irp_integration import IRPClient
from helpers.constants import BatchType, BatchStatus

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_02_Group_Rollup.ipynb')

# Display context
ux.header("Group Rollup Batch (Groups of Groups)")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"Step tracking initialized for '{context.step_name}'")

## 2) Verify Prerequisites

In [ ]:
# Verify that Grouping batch (Step_01) is complete
ux.header("Prerequisite Check")
ux.info("Rollup groups contain references to other groups.")
ux.info("Those groups must be created first (via Step_01).")
ux.info("")

# Query for Stage_01/Step_03 step run to get batch IDs
query = """
    SELECT sr.id, sr.step_id, sr.run_num, sr.output_data, sr.completed_ts
    FROM irp_step_run sr
    INNER JOIN irp_step s ON sr.step_id = s.id
    INNER JOIN irp_stage sg ON s.stage_id = sg.id
    INNER JOIN irp_cycle c ON sg.cycle_id = c.id
    WHERE c.cycle_name = %s
      AND sg.stage_num = 1
      AND s.step_num = 3
      AND sr.status = 'COMPLETED'
    ORDER BY sr.completed_ts DESC
    LIMIT 1
"""

result = execute_query(query, (context.cycle_name,))

if result.empty:
    raise ValueError("Batch creation step not found - please complete Stage_01/Step_03 first")

output_data = result.iloc[0]['output_data']
batches = output_data.get('batches', {})

# Check for GROUPING_ROLLUP batch
if BatchType.GROUPING_ROLLUP not in batches:
    ux.warning("No Grouping Rollup batch found in this cycle.")
    ux.info(f"Available batches: {list(batches.keys())}")
    ux.info("")
    ux.info("This cycle may not have any rollup groups (groups of groups).")
    ux.info("If you expected rollup groups, check your configuration file.")
    step.complete({'skipped': True, 'reason': 'No rollup batch in cycle'})
    raise SystemExit("No Grouping Rollup batch - step not needed")

# Check if GROUPING batch exists and is complete
if BatchType.GROUPING in batches:
    grouping_batch_id = int(batches[BatchType.GROUPING])
    grouping_batch = read_batch(grouping_batch_id)
    grouping_status = grouping_batch['status']
    
    ux.subheader("Grouping Batch (Step_01) Status")
    ux.info(f"Batch ID: {grouping_batch_id}")
    ux.info(f"Status: {grouping_status}")
    
    if grouping_status != BatchStatus.COMPLETED:
        ux.error("")
        ux.error(f"Grouping batch is {grouping_status}, not COMPLETED")
        ux.error("Rollup groups cannot be created until the base groups exist.")
        ux.error("")
        ux.error("Please complete Step_01_Group_Analysis_Results.ipynb first,")
        ux.error("then return to this notebook.")
        step.fail(f"Prerequisite not met: Grouping batch is {grouping_status}")
        raise ValueError(f"Grouping batch must be COMPLETED before running rollup. Current status: {grouping_status}")
    
    ux.success("Grouping batch is COMPLETED - child groups exist")
else:
    ux.info("No Grouping batch in this cycle (rollup groups may reference only analyses)")

ux.success("")
ux.success("Prerequisites verified - ready to proceed")
step.log("Prerequisites verified: Grouping batch complete")

## 3) Retrieve Grouping Rollup Batch

In [ ]:
# Retrieve Grouping Rollup batch
ux.subheader("Retrieve Grouping Rollup Batch")

rollup_batch_id = int(batches[BatchType.GROUPING_ROLLUP])

ux.success(f"Retrieved Grouping Rollup batch: ID={rollup_batch_id}")
step.log(f"Retrieved Grouping Rollup batch: ID={rollup_batch_id}")

## 4) Review Grouping Rollup Configuration

In [ ]:
# Verify batch status and display job information
ux.subheader("Verify Batch Status")

# Read batch details
batch = read_batch(rollup_batch_id)

batch_info = [
    ["Batch ID", batch['id']],
    ["Batch Type", batch['batch_type']],
    ["Status", batch['status']],
    ["Created", batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(batch_info, headers=["Property", "Value"])

# Get jobs in batch
jobs = get_batch_jobs(rollup_batch_id)
job_count = len(jobs)

ux.info(f"\nTotal rollup jobs: {job_count}")

# Show sample configurations
if job_count > 0:
    ux.info("\nRollup Group Configurations:")
    rows = []
    for i, job in enumerate(jobs[:10]):
        config_query = "SELECT job_configuration_data FROM irp_job_configuration WHERE id = %s"
        config_result = execute_query(config_query, (job['job_configuration_id'],))
        if not config_result.empty:
            config = config_result.iloc[0]['job_configuration_data']
            
            # Get group details
            group_name = config.get('Group_Name', 'N/A')
            items = config.get('items', [])
            items_count = len(items)
            # Show first few items as preview
            items_preview = ', '.join(items[:3])
            if len(items) > 3:
                items_preview += f', ... (+{len(items) - 3} more)'
            
            rows.append([
                group_name,
                items_count,
                items_preview
            ])
    ux.table(rows, headers=["Group Name", "# Items", "Items (Preview)"])
    
    if job_count > 10:
        ux.info(f"... and {job_count - 10} more rollup job(s)")

step.log(f"Verified batch: {job_count} rollup jobs ready for submission")

## 5) Submit Grouping Rollup Batch to Moody's

In [ ]:
# Submit batch to Moody's API
ux.subheader("Submit Rollup Batch to Moody's")

ux.info("")
ux.info("Submission Process:")
ux.info("  - Each job will group the specified items (groups and/or analyses)")
ux.info("  - Item names will be resolved to URIs (groups created in Step_01 now exist)")
ux.info("  - Rollup results will be created in Moody's Risk Modeler")
ux.info("  - Jobs will transition to SUBMITTED status")
ux.info("  - Batch will transition to ACTIVE status")
ux.info("")

# Submit
ux.info("\nSubmitting batch...")

# Pass step.step_id to associate batch with this step (not the creation step)
result = submit_batch(rollup_batch_id, IRPClient(), step_id=step.step_id)

# Display results
ux.success(f"\nBatch submission completed")
ux.info(f"  Submitted: {result['submitted_jobs']} jobs")
ux.info(f"  Status: {result['batch_status']}")

# Check for errors
failed_count = len([j for j in result['jobs'] if 'error' in j])
if failed_count > 0:
    ux.warning(f"\n{failed_count} job(s) failed to submit")
    for job_result in result['jobs']:
        if 'error' in job_result:
            ux.error(f"  Job {job_result['job_id']}: {job_result['error']}")

step.log(f"Rollup batch submitted: {result['submitted_jobs']} jobs, {failed_count} failed")

## 6) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

try:
    # Prepare output data
    output_data = {
        'batch_id': rollup_batch_id,
        'batch_type': batch['batch_type'],
        'batch_status': result['batch_status'],
        'submitted_jobs': result['submitted_jobs'],
        'failed_jobs': failed_count
    }
    
    # Complete the step
    step.complete(output_data)

    ux.success("\n" + "="*60)
    ux.success("GROUPING ROLLUP BATCH SUBMITTED SUCCESSFULLY")
    ux.success("="*60)
    ux.info(f"\nSubmitted {result['submitted_jobs']} rollup job(s) to Moody's API")
    if failed_count > 0:
        ux.warning(f"{failed_count} job(s) failed to submit")
    ux.info(f"Batch status: {result['batch_status']}")
    ux.info("\nNext: Monitor job progress or proceed to Export stage")

except Exception as e:
    ux.error(f"Step completion failed: {str(e)}")
    step.fail(str(e))
    raise